# SVM

Instalando requisitos

In [1]:
%pip install -r requirements.txt -q
%pip install pandas -q
%pip install scikit-learn -q
%pip install tensorflow -q
%pip install numpy -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importando requisitos

In [2]:
from app.modelo.imagem_rgb import ImagemRGB
import sklearn.model_selection
import pandas
import tensorflow
import os
import numpy
import pickle
import random
from sklearn import svm

2024-06-05 10:32:33.329055: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 10:32:41.891195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Definindo variáveis

In [3]:
image_directory = os.path.join(os.getcwd(), 'dataset_converted')
num_classes = 6

Abrindo base de dados

In [4]:
data_frame = pandas.read_csv("classifications.csv")

Adicionando coluna path

In [5]:
for index, row in data_frame.iterrows():
    data_frame.at[index, 'path'] = os.path.join(image_directory, row['bethesda_system'], "{}.png".format(row['cell_id']))

In [6]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path
0,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,1,SCC,792,462,/mnt/home/henrique/Repositorios_Git/GitHub.com...
1,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,2,SCC,601,678,/mnt/home/henrique/Repositorios_Git/GitHub.com...
2,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,3,SCC,363,467,/mnt/home/henrique/Repositorios_Git/GitHub.com...
3,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,4,SCC,599,437,/mnt/home/henrique/Repositorios_Git/GitHub.com...
4,400,9ae8a4edde40219bad6303cebc672ee4.png,10.6084/m9.figshare.12230906,5,Negative for intraepithelial lesion,1186,450,/mnt/home/henrique/Repositorios_Git/GitHub.com...
...,...,...,...,...,...,...,...,...
11529,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11530,LSIL,618,407,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11530,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11531,LSIL,607,374,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11531,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11532,LSIL,707,251,/mnt/home/henrique/Repositorios_Git/GitHub.com...
11532,1,be340ee72689dfe3f8dc9c24de6127f4.png,10.6084/m9.figshare.12229511,11533,LSIL,579,246,/mnt/home/henrique/Repositorios_Git/GitHub.com...


Adicionando coluna random e ordenando por meio dela

In [7]:
data_frame['random'] = data_frame[data_frame.columns[0]].apply(lambda _: random.random())

In [8]:
data_frame.sort_values(
    by='random',
    inplace=True
)

Removendo imagens que não existem

In [9]:
for index, row in data_frame.iterrows():
    if not os.path.exists(row['path']):
        data_frame.drop(index, inplace=True)

In [10]:
data_frame

,image_id,image_filename,image_doi,cell_id,bethesda_system,nucleus_x,nucleus_y,path,random
2514,318,a30fcaf343b17a21a7018e1ed1ec907e.png,10.6084/m9.figshare.12230615,2515,Negative for intraepithelial lesion,690,637,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000469
8185,82,dd61f345bc7cdcbf3bb86d58eb0992b4.png,10.6084/m9.figshare.12229778,8186,Negative for intraepithelial lesion,759,695,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000521
10804,25,f981c4aac622d7cce71a81305dd38ade.png,10.6084/m9.figshare.12229598,10805,Negative for intraepithelial lesion,948,591,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.000976
7680,97,299abd9ed2593fa2a0bcd9ba0c73a576.png,10.6084/m9.figshare.12229847,7681,ASC-US,1094,733,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.001125
1551,358,e0c93ad46ad5dac7832427c1eb191c9e.png,10.6084/m9.figshare.12230747,1552,Negative for intraepithelial lesion,474,118,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.001143
...,...,...,...,...,...,...,...,...,...
8645,72,424a7ee29385c53e950de973f411a30c.png,10.6084/m9.figshare.12229742,8646,Negative for intraepithelial lesion,660,791,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999164
6477,136,8cfdeaf76c0a48043406626f1a0b90a7.png,10.6084/m9.figshare.12230009,6478,Negative for intraepithelial lesion,1176,270,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999224
10552,30,0489736e8d880b5188b04d11c3131cd3.png,10.6084/m9.figshare.12229613,10553,Negative for intraepithelial lesion,834,918,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999734
10954,19,8457af003b1ad48e907156b5e6e53bea.png,10.6084/m9.figshare.12229580,10955,Negative for intraepithelial lesion,742,394,/mnt/home/henrique/Repositorios_Git/GitHub.com...,0.999864


Separando o x

In [11]:
data_frame['path'].iloc[0]

'/mnt/home/henrique/Repositorios_Git/GitHub.com/Henriquemcc/Ciencia da Computacao/Trabalho_Pratico_-_Processamento_e_Analise_de_Imagens_-_2024-1/develop_henrique_2024_06_01/dataset_converted/Negative for intraepithelial lesion/2515.png'

In [12]:
xi = ImagemRGB.from_file(data_frame['path'].iloc[0]).matriz.flatten()
xi

array([158, 114, 108, ..., 193, 196, 220], dtype=uint8)

In [13]:
xi.shape

(30000,)

In [14]:
x = []
for index, row in data_frame.iterrows():
    image = ImagemRGB.from_file(row['path'])
    x.append(image.matriz.flatten()/255)
x = numpy.array(x)

In [15]:
x.shape

(5581, 30000)

In [16]:
x

array([[0.61960784, 0.44705882, 0.42352941, ..., 0.75686275, 0.76862745,
        0.8627451 ],
       [0.88627451, 0.57647059, 0.31372549, ..., 0.80392157, 0.73333333,
        0.76470588],
       [0.25490196, 0.16862745, 0.26666667, ..., 0.18823529, 0.0745098 ,
        0.14117647],
       ...,
       [0.82745098, 0.44313725, 0.2       , ..., 0.        , 0.        ,
        0.        ],
       [0.85490196, 0.59215686, 0.52941176, ..., 0.10196078, 0.1372549 ,
        0.27843137],
       [0.94509804, 0.61568627, 0.64705882, ..., 0.97647059, 0.99215686,
        0.99215686]])

Separando o y

In [41]:
y = data_frame.iloc[:, 4].values
#label_encoder = sklearn.preprocessing.LabelEncoder()
#y = label_encoder.fit_transform(y)

In [42]:
labels = list(set(data_frame.iloc[:, 4].values))

In [43]:
labels

['SCC',
 'HSIL',
 'Negative for intraepithelial lesion',
 'LSIL',
 'ASC-H',
 'ASC-US']

In [44]:
#y = [[1 if label==lbl else 0 for lbl in labels] for label in y]

In [45]:
#y = numpy.array(y)

In [46]:
y

array(['Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion', ...,
       'Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion'], dtype=object)

In [47]:
type(y)

numpy.ndarray

In [48]:
y.shape

(5581,)

Separando o conjunto de treino e teste

In [49]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, train_size=0.8)

In [50]:
y_train

array(['Negative for intraepithelial lesion', 'LSIL',
       'Negative for intraepithelial lesion', ...,
       'Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion',
       'Negative for intraepithelial lesion'], dtype=object)

In [51]:
x_train.shape

(4464, 30000)

In [52]:
y_train.shape

(4464,)

In [53]:
x_test.shape

(1117, 30000)

In [54]:
y_test.shape

(1117,)

Exportando x_test, x_train, y_test, y_train para um arquivo externo

In [55]:
#with open('data.pkl', mode='wb') as file:
    #pickle.dump([x_train, x_test, y_train, y_test], file)

# Aplicando o SVM

Construindo o modelo

In [56]:
modelo = svm.SVC()

Treinando o modelo

In [57]:
modelo.fit(x_train, y_train)

In [ ]:
import joblib
joblib.dump(modelo, 'modelo.pkl')

In [ ]:
with open('modelo.pkl', mode='wb') as file:
    pickle.dump(modelo, file)